# Item 22. 일반적인  알고리즘을 구현할 때, 제네릭을 사용해라

## 제네릭?

특정 타입에 의존하지 않고 하나의 값이 여러 타입이 될수 있음을(제네릭으로 지정한) 기술에 중점을 두어 재사용성을 높이는 프로그래밍 방식

- 자바에서는 1.5부터 도입


## 제네릭을 완벽하게 이해하기 위해 알아두어야 할것들! - Deep dive

### 기본
- 타입 파라미터(매개변수) : 제네릭 타입인 파라미터
`class Box<T>(val item: T) { /* ... */ }` 같이 표현

- 타입 인수 : 실제 파라미터에 들어가는 값, 인수
- 제네릭 클래스 : 타입 매개변수가 정의된 클래스
- 제네릭 함수 : 타입 매개변수가 정의된 함수

### Variance, Invariance, Contravariance

- 통계학에서 variance는 분산, covariance는 공분산 ... 이지만 컴퓨터 공학(cs)에서는 전혀 다른 의미

- 타입들간의 서브타입 슈퍼타입 계층화를 표현할때 사용한다.


Variance : 공변
Invariance : 불공변
Contravariance : 반공변

`공변의 의미는 A ⊂ B 일때 고차함수 T에 대해 T<A> ⊂ T<B> 로 그 관계가 유지된다면 공변이라 표현한다.`

`불공변의 의미는 A ⊂ B 일때 고차함수 T에 대해 T<A> -관계가 없음- T<B> 으로 정의된 관계가 사용되 고차함수상 아무 의미를 가지지 않을때 불공변이라 표현한다`

`반공변의 의미는 A ⊂ B 일때 고차함수 T에 대해 T<B> ⊂ T<A> 로 그 관계가 역전된다면 반공변이라 표현한다.`



### 리스코프 치환원칙 (LSP)

- SOLID 의 L

정의는 아래와 같다.

- 하위형에서 반환형의 공변성
- 하위형에서 메서드 인수의 반공변성

하지만 이 리스코프 치환원칙을 많은 프로그래밍 언어들은 제대로 지키지 않는다.

### 자바, 코틀린에서 리스코프 치환원칙 평가해보기

#### 메서드 반환 타입 - LSP 지킴

In [ ]:
abstract class Parent {
    public abstract fun foo(a: Int): Number
}

class Child: Parent() {
    override fun foo(a: Int): Int {
        return a + 10
    }
}

val p: Parent = Child()
val b: Number = p.foo(10)

부모의 함수 foo 의 반환값은 Number 타입이나 오버라이딩으로 반환값을 더 낮은 타입인 Int로 변경했다.

> 하위타입 **Child**의 함수 foo()의 반환타입 **Int** **⊂** 상위 타입 **Parent**의 함수 foo()의 반환타입 **Number**

위의 의미는 자바, 코틀린에서 함수의 반환타입은 공변을 갖는다는 의미다.

즉 리스코프 치환원칙 1항의
- 하위형에서 반환형의 공변성
는 만족한다.


#### 메서드 파라미터 타입 - LSP 안지킴!

In [1]:
abstract class Parent {
    public abstract fun foo(bar: Number)
}

class CovariantChild: Parent() {
    override fun foo(bar: Int) {} // 컴파일 에러!!! 공변 아님.
}

class ContravriantChild: Parent() {
    override fun foo(bar: Any) {} // 컴파일 에러!!! 반공변도 아님
}

val p: Parent = ContravriantChild()
p.foo(10)

//Line_1.jupyter.kts (5:1 - 21) Class 'CovariantChild' is not abstract and does not implement abstract base class member public abstract fun foo(bar: Number): Unit defined in Line_1_jupyter.Parent
//Line_1.jupyter.kts (6:5 - 13) 'foo' overrides nothing
//Line_1.jupyter.kts (9:1 - 24) Class 'ContravriantChild' is not abstract and does not implement abstract base class member public abstract fun foo(bar: Number): Unit defined in Line_1_jupyter.Parent
//Line_1.jupyter.kts (10:5 - 13) 'foo' overrides nothing

Line_1.jupyter.kts (5:1 - 21) Class 'CovariantChild' is not abstract and does not implement abstract base class member public abstract fun foo(bar: Number): Unit defined in Line_1_jupyter.Parent
Line_1.jupyter.kts (6:5 - 13) 'foo' overrides nothing
Line_1.jupyter.kts (9:1 - 24) Class 'ContravriantChild' is not abstract and does not implement abstract base class member public abstract fun foo(bar: Number): Unit defined in Line_1_jupyter.Parent
Line_1.jupyter.kts (10:5 - 13) 'foo' overrides nothing

오버라이딩을 통해 하위타입 Child의 함수 foo 파라미터를 어떤걸로 변경해도(상위, 하위 둘다) 컴파일 에러가 발생한다.

즉 리스코프 치환원칙의 2항
- 하위형에서 메서드 인수의 반공변성

을 지키지 않고 불공변성을 가지고 있다.


#### 일급함수의 경우 - LSP 준수

In [ ]:

// 자식의 input이 Number, output이 Number일경우
var sub: (Number) -> Number = { i ->
    i
}
// 고차함수를 업캐스팅 해보면
var super1: (Number) -> Any = sub // 업캐스팅 타입의 output이 Number의 상위타입이므로 공변!
var super2: (Int) -> Number = sub // 업캐스팅 타입의 input이 Number의 하위타입이므로(Int) 반공변!
var super3: (Int) -> Any = sub // 둘다 적용도 가능

다형성을 위한 오버로딩의 존재때문에 클래스간의 input 반공변은 안지켜졌지만

일급함수의 경우는 오버로딩이 없기때문에 LSP가 준수된다.


#### 고차함수의 경우

In [ ]:
// 일급함수 정의
var sub: (Number) -> Number = { i ->
    i
}
var super1: (Number) -> Any = sub
var super2: (Int) -> Number = sub
var super3: (Int) -> Any = sub
var super4: (Any) -> Any = { i ->
    i
}


// 고차함수로 일급함수를 다뤄보면?
fun higher(block: (Number) -> Any) {} // input을 super1 일급함수로 정의했지만

// 고차함수로 일급함수들을 사용해보면?
higher(sub) // sub은 super1의 하위타입고 input은 반공변이므로 하위타입이 가능!
higher(super1) // 자기자신이므로 당연히 된다
//higher(super2) // super1과 2는 다른타입이므로 안된다
//higher(super3) // super1과 3은 다른 타입이므로 안된다
higher(super4) // 일급함수 (Number) -> Any 과 (Any) -> Any를 비교해보면 input이 반공변이여야하므로 Number 인풋이 더 상위타입으로 정의가능하다.
// 즉 (Any) -> Any ⊂ (Number) -> Any 이고 super4는 super 1의 하위타입이 되므로 하위타입을 인자로 받을수 있게된다.

super1 = super4 // 따라서 이렇게 업캐스팅도 됨

#### 배열 - 다형성의 문제

자바는 최초 언어 설계시 타입만 존재했기때문에(제네릭이 없었으므로) 배열 설계에 많은 고민이 있었고, 일부 결점이 존재한다.

일단 배열은 특정 타입을 다루어야한다.
따라서 배열의 공통 API는 재사용성이 높을수밖에없다. 이 배열의 공통 API를 재사용하려면 상속구조를 통해 다형성으로 푸는게 가장 기본적인 접근이였고, 자바도 배열을 상위 하위 계층화된 타입으로 정의하였다.

In [ ]:
Integer[] numbers = {1,4,2,1};
Object[] objects = numbers; // output은 covariant니까 된다.
objects[2] = "B"; // input을 하면 런타임오류. ArrayStoreException

하지만 자바는 리스코프 치환원칙을 절반만 지키는 반쪽짜리였기때문에 상위타입으로 치환 후 input의 반공변성은 지켜지지 않으므로 치환되지 않는 문제가 발생한다.

이 치환불가 문제를 런타임 오류로 핸들링하였고 이는 타입 안정성의 불안을 낳았다.
- https://en.wikipedia.org/wiki/Covariance_and_contravariance_(computer_science)#Covariant_arrays_in_Java_and_C#

즉
- 자바의 타입은 리스코프 치환원칙을 부분적으로만 따름
- 자바의 배열은 재사용을 위해 다형성을 원했고, 계층화된 타입으로 정의되있음(Object[], Integer[])
- 리스코프 치환원칙을 절반만 따름으로서 리턴타입의 공변(타입캐스팅)은 가능하나, 인풋 타입의 반공변은 지키지 않아 런타입 에러로 이를 방어하였고 불안정한 핸들링임



코틀린의 경우도 역시 타입은 자바 베이스(jvm 베이스)이며 리스코프 치환원칙을 부분적으로만 따른다.

따라서 배열을 계층화된 타입으로 정의하면 같은 문제가 발생하는데 이를 Array<T>로 하나의 타입으로만 정의하여 해결했다.

이는 사실 List와 동일한 방식인데, 제네릭이 존재하는 시점에 설계되었기때문에 가능한 방식이다.

즉
- 코틀린의 타입은 리스코프 치환원칙을 부분적으로만 따름(자바와 동일)
- 코틀린의 배열은 하나의 타입만 사용해 재사용을 해결함
- 타입이 하나이므로 리스코프 치환원칙을 고려할필요가 없음(불공변성)
- 타입 불안정의 문제는 제네릭으로 해결

#### 제네릭과 리스코프 치환원칙

위의 배열 내용을 정확히 이해하기 위해서는 제네릭을 보다 확실히 알아야한다.

기본적으로 자바, 코틀린의 제네릭은 불공변성을 가진다.

이는 리스코프 치환원칙의 두 항목
- 하위형에서 반환형의 공변성
- 하위형에서 메서드 인수의 반공변성

이 둘을 정확히 표현하지 못하기 때문에 차라리 불공변성으로 처리하자는 의도이다.

In [27]:
class Invariant<T> {
    val foo: T

    constructor(foo: T) {
        this.foo = foo
    }

    fun bar(): T {
        return foo
    }
}

val invariant1 = Invariant<Int>(3)
val invariant2: Invariant<Number> = invariant1 // invariant이므로 불가능

Line_27.jupyter.kts (14:37 - 47) Type mismatch: inferred type is Line_27_jupyter.Invariant<Int> but Line_27_jupyter.Invariant<Number> was expected


하지만 만약 해당 타입이 output에서만 사용된다면 공변성을 갖는 타입으로 정의도 가능하다.

In [28]:
class Variant<out T> {
    val foo: T

    constructor(foo: T) {
        this.foo = foo
    }

    fun bar(): T {
        return foo
    }
}


val Variant1 = Variant<Int>(3)
val Variant2: Variant<Number> = Variant1

위와같이 제네릭 파라미터 T가 out으로만 사용될것이라고 명시해주면(`out T`) 이는 리스코프 치환원칙에서 문제가 생길일이 없기때문에 Variant로서 Variant<Number>가 Variant<Int>의 상위형이 되고 업캐스팅 가능해진다.


In [ ]:
class ContraVariant<in T> {
    fun foo(foo : T) {
        println(foo)
    }
}


val ContraVariant1 = ContraVariant<Number>()
val ContraVariant2: ContraVariant<Int> = ContraVariant1

반대로  제네릭 파라미터 T가 input으로만 사용한다면 리스코프 치환원칙에 따라 input은 반공변을 가지므로 ContraVariant<Int>가 ContraVariant<Number>의 상위형이 되고 ContraVariant<Number>에서 ContraVariant<Int> 로 업캐스팅이 가능해진다.

즉 리스코프 치환원칙을 준수하기위해 위와같은 전략을 취했다고 볼 수있다.

### 제네릭 위치 변성

위에서 알아본것처럼 제네릭 선언시 그 변성(in or cont or va)을 정할수도 있지만, 제네릭을 사용할때 지정할수도 있다.

이를 제네릭 사용위치 변성이라 부르며


In [ ]:
val aa: Invariant<out Any> = Invariant<Int>(10)


위와같이 사용도 가능하다.

### 자바에서의 extend super?

자바의 제네릭 선언방식인 extend super는 bounded 의미도 겸하고있으나 코틀린은 이를 <out T : Number> 와 같이 분리하여 표현한다.